In [30]:
# prepare packages
import requests, pandas as pd
from datetime import datetime, timedelta, timezone
from bs4 import BeautifulSoup
import logging
# from pythonjsonlogger import jsonlogger
#import logmatic
import os
from jieba.analyse import extract_tags
import sys
import logstash
import time, random
import pymongo
import jieba , re



現在處理:  http://www.ptt.cc/bbs/Food/M.1127916153.A.21C.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1127927104.A.DD1.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1127964524.A.A08.html
進入ptt :  https://www.ptt.cc/bbs/Food/index57.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1128690757.A.92B.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1128692867.A.9B9.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1128734714.A.216.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1127579720.A.F1A.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1128756706.A.CA4.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1127580135.A.25C.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1128787804.A.A49.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1127612026.A.2EB.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1128803416.A.092.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1127612403.A.10B.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1126968978.A.984.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1126970021.A.D53.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1126972726.A.D63.html
現在處理:  http://www.ptt.

In [16]:
# 建立 MongoClient
from Mongo_account import MongoBase
host = '10.120.14.105'
client = "mongodb://" + host + ":27017/"
# 多執行續下 可能被 dead lock 
myclient = pymongo.MongoClient(client,
                               username = MongoBase.username,
                               password = MongoBase.password,
                               authSource = MongoBase.authSource,
                               authMechanism= MongoBase.authMechanism,
                               connect=False)
# 選擇 DB
mydb = myclient["cb104_G3"]
# 選擇 collection
mycol = mydb["Ptt_fix_security"]

jieba.load_userdict("bigdict.txt")


# insert many to mongo
# x = mycol.insert_many(mylist)
# many should be a list of dict

# 傳回 _id 的值
# print(x.inserted_ids)

In [18]:
# memory
#url = "https://www.ptt.cc/bbs/Food/index2.html"

# set proxy to use
proxylist = [{"http": "37.187.120.123:80"},            
             {"https": "178.128.31.153:8080"},
             {"http": "167.114.180.102:8080"},
             {"https" : "218.48.229.153:808"}, 
             {"http" : "85.30.219.120:46761"},
             {"http" : "183.101.103.164:8080"},
             {"http": "167.114.196.153:80"},
            {"http":"220.132.207.11:45069"}]
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}


In [26]:
# 直接傳進 Logstash 在進入 Elasticsearch


logger = logging.getLogger('python-logstash-logger')
logger.setLevel(logging.INFO)
host_demo = "logstash"
logger.addHandler(logstash.TCPLogstashHandler(host_demo,5959))






In [20]:
# 製作 url List
def get_url(n=2,m=3):
    result = []
    # 增加直覺 從第n頁 到 第m頁
    m = m + 1
    for t in range(n,m):
        url = "https://www.ptt.cc/bbs/Food/index" + str(t) + ".html"
        result.append(url)
    return result

8

In [21]:
# slove proxy connect error try next
def get_connect(url):
    i = random.randint(0,len(proxylist)-1)
    
    while True :
        
        try :
            resp= requests.get(url,headers=header,proxies=proxylist[i],timeout=120)          
            resp.encoding = "utf-8"
            time.sleep(random.randint(1,10)/10)
            logger.info("Request response",exc_info=True,extra={"url" : url, "time_use" : resp.elapsed.total_seconds(), "Response code" : resp.status_code, "proxy_use":proxylist[i]})
           
            
        except requests.exceptions.ConnectionError :             
            logger.warning("Proxy Error",exc_info=True,extra={"proxy":proxylist[i],"url":url})
            
            i = (i + 1) % len(proxylist)
            continue
        except requests.exceptions.ConnectTimeout :
            logger.warning("Connect Time out",exc_info=True)
            i = (i + 1) % len(proxylist)
            continue

            
            
        
        if resp.status_code == requests.codes.ok :
            code = "HTTP response code = " + str(resp.status_code)
            
                
            break        
        if resp.status_code == 404 :
            logger.warning("This page is gone nowhere !",exc_info=True,extra={"url":url})
                
            break
        i = i + 1
    
        
    return resp

In [22]:
# 自定義 404 connectError
class Connect404Except(Exception):
    pass

def parse(urls):
    # 進入ptt列表 抓取 標題、推文數、網址
    result = []
    start = time.time()
    
    connect = get_connect(urls)
    try :
        if connect.status_code == 404:
            raise Connect404Except
            
        else :
            soup = BeautifulSoup(connect.text)
            
    except Connect404Except : 
        logger.error("index error",exc_info=True,extra={"index":urls})
        pass
    end_index = time.time()   
    logger.info("index request complete",exc_info=True,extra={"time_use":end_index-start})
    if soup :    
        print("進入ptt : ", urls)
        post = soup.find_all("div", class_="r-ent")
        
        
        for r in post:
            
            if r.find("div", class_="nrec").text:
                try:
                    push = int(r.find("div", class_="nrec").text)
                except ValueError:
                    pass
            if r.find("a"):
                # 取得網址
                href = "http://www.ptt.cc" + r.find("a")["href"]
                # 取得標題
                title = r.find("a").text

                # 進入第二層 進入每一篇
                # 去除公告 請益 為分類的文章
                if "公告" in title:
                    continue
                elif "問卷" in title :
                    continue
                else :    
                    print("現在處理: ", href)

                    article = get_connect(href)
                    try:
                        if article.status_code == 404:
                            raise Connect404Except
                        else:
                            art = BeautifulSoup(article.text)  

                    except Connect404Except:
                        logger.error("Page error",exc_info=True,extra={"index":urls})
                        continue

                    end_post_request = time.time()
                    logger.info("post request complete",exc_info=True,extra={"time_use":end_post_request-end_index})
                    time.sleep(random.randint(1,10)/10)



                    if art:
                        content = art.find("div", id="main-content")


                        # 保存要丟棄的資訊
                        val = content.find_all("span", {"class": "article-meta-value"})
                        #  文章排版問題 有些文章沒有部份以下資訊
                        try:
                            author = val[0].text

                            date_in_content = val[3].text
                        except IndexError as e:
                            logger.warning("article span list out of index",exc_info=True, extra={"url": href})
                            author = ""
                            date_in_content = ""

                        # 開始丟棄資訊
                        removes = content.find_all("div", class_="article-metaline")
                        for remove in removes:
                            remove.extract()
                        removes = content.find_all("div", class_="article-metaline-right")
                        for remove in removes:
                            remove.extract()
                        removes = content.find_all("span", class_="f2")
                        removes = content.find_all("span", class_="f6")
                        for remove in removes:
                            if "※" in remove.text:
                                remove.extract()
                        # 處理推噓文
                        ps = content.find_all("div", class_="push")
                        score = 0
                        push = 0
                        pos = []
                        neg = []
                        arrow = []
                        if ps:
                            for p in ps:
                                tag = p.find("span", class_="push-tag").text
                                if "推" in tag:
                                    score = score + 1
                                    # replace 第三個參數 只置換第一個
                                    # push_content = p.find("span", class_="push-content").text.replace(": ","",1)
                                    push_content = p.find("span", class_="push-content").text
                                    push_ID = p.find("span", class_="push-userid").text
                                    push_total = push_ID + push_content
                                    pos.append(push_total)
                                elif "噓" in tag:
                                    score = score - 1
                                    push_content = p.find("span", class_="push-content").text
                                    push_ID = p.find("span", class_="push-userid").text
                                    push_total = push_ID + push_content
                                    neg.append(push_total)
                                else:
                                    push_content = p.find("span", class_="push-content").text
                                    push_ID = p.find("span", class_="push-userid").text
                                    push_total = push_ID + push_content
                                    arrow.append(push_total)
                                p.extract()


                        try:
                            content = content.text.replace("\r", "").replace("\n", "").replace(" ","")
                            content = re.sub(r"\W+","",content)
                        except AttributeError as e:
                            logger.warning("content text cant regularize",exc_info=True, extra={"url": href})




                        result.append({
                            "title": title,
                            "author": author,
                            "date": date_in_content,
                            "push": push,
                            "href": href,
                            "content": content,
                            "score": score,
                            "推文": pos,
                            "噓文": neg,
                            "箭頭": arrow,
                            "tags" : extract_tags(content , 10)

                        })
                        end_post = time.time()
                        logger.info("post complete",exc_info=True,extra={"time_use":end_post-end_post_request})
        
        # 進入 db
        try:
            end_all = time.time() - start
            # insert many ordered = false -> collection中有 唯一鍵值 若遇上duplicate 則跳過
            x = mycol.insert_many(result, ordered=False)
            if x.inserted_ids :
                logger.info("Insert Successful",exc_info=True,extra={"count":len(x.inserted_ids),"index_url":urls,"time_use":end_all})
        
        except pymongo.errors.BulkWriteError as e:
            
            for excep in e.details['writeErrors'] :
                
                logger.warning("DB error",exc_info=True,extra={"error_detail":excep["errmsg"], "index_url" : urls})
            
            
            
        




    return result

In [ ]:
logger.info("test",exc_info=True)

In [29]:
from concurrent.futures import ThreadPoolExecutor, wait, ProcessPoolExecutor, as_completed

new_result = []
with ThreadPoolExecutor(max_workers= 10,thread_name_prefix="ptt_crawler") as executor:
    future_result = {executor.submit(parse, url) : url for url in get_url(21,2000)}

    

for future in as_completed(future_result):
        
    logger.info("Worker's job done",exc_info=True)
        
    
    

進入ptt :  https://www.ptt.cc/bbs/Food/index24.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1103921101.A.762.html
進入ptt : 進入ptt :  https://www.ptt.cc/bbs/Food/index21.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1100937947.A.F55.html
 https://www.ptt.cc/bbs/Food/index26.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1105113979.A.EC7.html
進入ptt :  https://www.ptt.cc/bbs/Food/index30.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1108642967.A.782.html
進入ptt :  https://www.ptt.cc/bbs/Food/index23.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1102616355.A.EFB.html
進入ptt :  https://www.ptt.cc/bbs/Food/index25.html
進入ptt :  https://www.ptt.cc/bbs/Food/index28.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1104420648.A.231.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1107570320.A.2BF.html
進入ptt :  https://www.ptt.cc/bbs/Food/index29.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1108256972.A.836.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1105206903.A.AFF.html
現在處理:  http://www.ptt.cc/bbs/Food/M.1108269840.A.708.html
現在處理:  http://www.pt

KeyboardInterrupt: 

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=["title","author","date","push","href","content","score","推文","噓文","箭頭"])
for r in new_result:
    for evey in r:
        s = pd.Series([evey["title"],evey["author"],evey["date"],evey["push"],evey["href"],evey["content"],evey["score"],evey["推文"],evey["噓文"],evey["箭頭"]]
                      ,index=["title","author","date","push","href","content","score","推文","噓文","箭頭"])
        df = df.append(s,ignore_index=True)
        

In [ ]:
df



In [ ]:
import json

collect = json.loads(df.T.to_json()).values()

In [ ]:
from pymongo import MongoClient

class MongoBase:
    def __init__(self,collection):
        self.collection=collection
        self.OpenDB()
    def OpenDB(self):
        user='root'
        passwd='root'
        # on local host
        host='192.168.114.130'
        port='27017'
        auth_db='admin'
        uri = "mongodb://"+user+":"+passwd+"@"+host+":"+port+"/"+auth_db+"?authMechanism=SCRAM-SHA-1"
        self.con = MongoClient(uri, connect=True)
        self.db=self.con['wangdong']
        self.collection=self.db[self.collection]
    def closeDB(self):
        self.con.close()
        
mongo = MongoBase('test3')

In [ ]:


mongo.collection.insert_many(collect)
mongo.closeDB()


In [ ]:
parse("https://www.ptt.cc/bbs/Food/index7004.html")

In [ ]:
# 進入ptt列表 抓取 標題、推文數、網址 
result = []
soup = BeautifulSoup(get_connect(url))
print("進入ptt : ",url)
post = soup.find_all("div", class_="r-ent")
for r in post:
            # 取得推的數量
    push = 0
    if r.find("div", class_="nrec").text:
        try:
            push = int(r.find("div", class_="nrec").text)
        except ValueError:
                pass
    if r.find("a"):
        # 取得網址
        href = "http://www.ptt.cc" + r.find("a")["href"]
        # 取得標題
        title = r.find("a").text
        
        # 進入第二層 進入每一篇
        # 去除公告 請益 為分類的文章
        if not "公告" in title :
            if not "請益" in title:
                print("現在處理: ", href)
                art = BeautifulSoup(get_connect(href))
                article = []
                content = art.find("div", id="main-content")
                # 保存要丟棄的資訊
                val = content.find_all("span", {"class":"article-meta-value"})
                author = val[0].text
                title_in_content = val[2].text
                date_in_content = val[3].text
                
                # 開始丟棄資訊
                removes = content.find_all("div", class_="article-metaline")
                for remove in removes:
                    remove.extract()
                removes = content.find_all("div", class_="article-metaline-right")
                for remove in removes:
                    remove.extract()
                removes = content.find_all("span", class_="f2")
                for remove in removes:
                    remove.extract()
                removes = content.find_all("div", class_="push")
                for remove in removes:
                    remove.extract()
                article.append({
                    "author" : author,
                    "title_in_content" : title_in_content,
                    "date_in_content" : date_in_content,
                    "content" : content.text
                })
                
    result.append({
        "title" : title,
        "push" : push,
        "href" : href,
        "content" : article
        
    })
    
    time.sleep(random.randint(1,6))
                






In [ ]:
import json
# 轉成 Dataframe
df = pd.DataFrame(result)



In [ ]:
# 排列欄位順序

df = df[ ["title", "push", "href", "content" ] ]

In [ ]:
df.iloc[1]

In [ ]:
# I/O 較多 使用 thread
start = time.time()
with ThreadPoolExecutor(max_workers=4) as executor:
    results = executor.map(int, ['1', '2', '3', '4', '5'])
    for v in results:
        print(v)
end = time.time()
print("time used :",end - start)